<a href="https://colab.research.google.com/github/vikramnarasimha/deeplearning-notebooks/blob/main/MNIST_Classify_Threes_And_Sevens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook 

     |████████████████████████████████| 727kB 11.2MB/s 
     |████████████████████████████████| 194kB 22.8MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 1.2MB 35.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 12.8MB 221kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [4]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/train'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/valid')]

In [5]:
def stacked_tensors(path,digit,training):
  training_or_validation = 'train'
  if(not training):
    training_or_validation = 'valid'
  digit_images = (path/training_or_validation/str(digit)).ls()
  digit_tensors = [tensor(Image.open(o)) for o in digit_images]
  stacked_digit_tensors = torch.stack(digit_tensors).float()/255
  return (digit_tensors, stacked_digit_tensors)

In [6]:
threes, stacked_threes = stacked_tensors(path,3, true)
sevens, stacked_sevens = stacked_tensors(path,7, true)
stacked_threes.shape, stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [7]:

train_x = torch.cat([stacked_threes.view(-1,(28*28)),
          stacked_sevens.view(-1,(28*28))]
)
train_x.shape



torch.Size([12396, 784])

In [8]:
train_y = tensor([1]* len(threes) + [0] * len(sevens)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [9]:
dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [10]:
def dset_3s_7s(path, training):
  '''
  Returns a dataset of the independent variable (image pixels) on the x
  and the category i.e which digit on the y (0 for 7 and 1 for 3)
  '''
  threes, stacked_threes = stacked_tensors(path,3,training)
  sevens, stacked_sevens = stacked_tensors(path,7,training)
  train_x = torch.cat([stacked_threes.view(-1,(28*28)),
          stacked_sevens.view(-1,(28*28))]
          )
  train_y = tensor([1]* len(threes) + [0] * len(sevens)).unsqueeze(1)
  #dset = list(zip(train_x, train_y))
  return train_x, train_y

In [11]:
train_x,train_y = dset_3s_7s(path, True)
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [12]:
valid_x,valid_y = dset_3s_7s(path, False)
valid_x.shape, valid_y.shape

(torch.Size([2038, 784]), torch.Size([2038, 1]))

In [13]:
#initialise weights
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

weights = init_params((28*28,1))
bias = init_params(1)
weights.shape, bias.shape

(torch.Size([784, 1]), torch.Size([1]))

In [14]:
#predict for 1 image
dset = list(zip(train_x, train_y))
x,y = dset[0]
new_y = (x*weights.T).sum() + bias
print(new_y, y)

#predict for all images using matrix multiplication @.. 
def linear1(xb):
  return xb@weights + bias
preds = linear1(train_x)
preds, train_y

#calculate loss
corrects = (preds > 0.0).float() == train_y
corrects 
print(corrects.float().mean())

#Gradient & Step
weights[0] *= 0.000000000001
preds = linear1(train_x)
corrects = (preds > 0.0).float() == train_y
corrects 
print(corrects.float().mean())



tensor([2.6166], grad_fn=<AddBackward0>) tensor([1])
tensor(0.5380)
tensor(0.5380)


# Do it yourself Model 

In [15]:
weights = init_params((28*28,1))
bias = init_params(1) 

#Using Dataloader to train 
dl = DataLoader(dset, batch_size=256)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape
params = weights, bias

#targets =tensor[1,0,1]
#predictions = tensor[0.9,0.4,0.3]
#loss = tensor[0.1,0.4,0.7]
def mnist_loss(predictions, targets):
  predictions = predictions.sigmoid()
  return torch.where(targets==1,1-predictions, predictions).mean()

def calc_grad(x, y, model):
    pred = model(x)
    loss = mnist_loss(pred,y)
    loss.backward()

def train_epoch(model, params, dl,lr):
  for x,y in dl:
    calc_grad(x,y,model)
    for p in params:
      p.data -= p.grad * lr
      p.grad.zero_()

def batch_accuracy(xb,yb):
  preds = xb.sigmoid()
  correct = (preds > 0.5) == yb
  return correct.float().mean()

def validate_epoch(model):
  accs = [batch_accuracy(model(xb),yb) for xb,yb in valid_dl]
  return round(torch.stack(accs).mean().item(),4)

lr = 0.0001
train_epoch(linear1, params, dl,lr)
validate_epoch(linear1)


0.5747

In [16]:
lr = 1. 
for i in range(20):
  train_epoch(linear1, params, dl, lr)
  print(validate_epoch(linear1), end = ' ')


0.7216 0.8592 0.9168 0.931 0.9422 0.951 0.9554 0.9583 0.9627 0.9656 0.9676 0.9691 0.9705 0.9715 0.972 0.973 0.974 0.9759 0.9769 0.9764 

## Model based on PyTorchs nn.Linear 

In [17]:
linear_model = nn.Linear(28*28,1)
class BasicOptimizer:
  def __init__(self,params, lr):
    self.params, self.lr = list(params),lr
  
  def step(self, *args, **kwargs):
    for p in self.params:
      p.data -= p.grad.data * self.lr
  
  def zero_grad(self, *args, **kwargs):
    for p in self.params:
      p.grad = None

lr = 1.
opt = BasicOptimizer(linear_model.parameters(), lr)

def train_epoch(model):
  for xb, yb in dl:
    calc_grad(xb,yb, model)
    opt.step()
    opt.zero_grad()

validate_epoch(linear_model)
    
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end = ' ')

train_model(linear_model, 20)

0.4932 0.7231 0.8608 0.917 0.9414 0.956 0.9643 0.9663 0.9678 0.9692 0.9717 0.9731 0.9751 0.976 0.976 0.9775 0.9775 0.978 0.9785 0.9785 

# Model based on FastAI's wrappers

In [21]:
linear_model = nn.Linear(28*28,1)
opt = SGD(linear_model.parameters(), lr)
dls = DataLoaders(dl, valid_dl)
learn = Learner(dls, linear_model, loss_func=mnist_loss, opt_func=SGD,metrics=batch_accuracy)
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.636948,0.502940,0.495584,00:00
1,0.381875,0.267059,0.746811,00:00
2,0.145848,0.161294,0.854760,00:00
3,0.067095,0.097924,0.918057,00:00
4,0.038013,0.070874,0.938665,00:00
5,0.026776,0.056389,0.954367,00:00
6,0.022078,0.047697,0.964181,00:00
7,0.019844,0.042124,0.966143,00:00
8,0.018614,0.038306,0.968106,00:00
9,0.017840,0.035525,0.969087,00:00
